In [40]:
from __future__ import print_function
import numpy as np
np.random.seed(1337)  # for reproducibility
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D, Average
from keras.utils import np_utils
from keras import backend as K
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

batch_size = 128
nb_classes = 2
nb_epoch = 10

# input image dimensions
img_rows, img_cols = 20, 20
# number of convolutional filters to use
nb_filters = 64
# size of pooling area for max pooling
pool_size = (2, 2)
# convolution kernel size
kernel_size = (3, 3)
input_shape=(20, 20,1)

In [41]:

vetor_atendente_final=pd.read_excel('Atendentes_Vetores.xlsx',header=0,index=False)
vetor_atendente_final.shape

(2771, 401)

In [42]:
vetor_cliente_final=pd.read_excel('Clientes_Vetores.xlsx',header=0,index=False)
vetor_atendente_final.iloc[:,0:-1].shape

(2771, 400)

In [43]:
target=np.array(pd.get_dummies(vetor_atendente_final.iloc[:,-1]))
target_=target
target

array([[0, 1],
       [0, 1],
       [0, 1],
       ...,
       [1, 0],
       [1, 0],
       [1, 0]], dtype=uint8)

In [44]:
def norm2(x):
    return (x-np.min(x))/(np.max(x)-np.min(x))

In [73]:
tabela_dados=pd.read_excel('TabelaDados.xlsx',header=0,index=False)
tabela_dados2=tabela_dados.iloc[np.concatenate([np.where(target.T[1]==1)[0][0:1680],np.where(target.T[1]==0)[0][-1091:]],axis=0),:].fillna(0.9)
tabela_dados2['Satisfação cliente']=tabela_dados2['Satisfação cliente'].astype('category').cat.codes
tabela_dados2['Satisfação atendente']=tabela_dados2['Satisfação atendente'].astype('category').cat.codes
tabela_dados3=tabela_dados2[['Silencio','Tempo de Ligação','Atend_Lex','Cliente_Lex','Satisfação cliente',
                                     'Satisfação atendente']]
tabela_dados3=norm2(tabela_dados3)

In [85]:
tabela_dados3.shape

(2771, 6)

In [76]:
for i in range(0,tabela_dados3.shape[1]):
    print(i,np.mean(tabela_dados3.iloc[np.where(target.T[1]==1)[0],i])-
            np.mean(tabela_dados3.iloc[np.where(target.T[1]==0)[0],i]))

0 0.04500811976191299
1 -0.02761422793258117
2 -0.14125480316922678
3 -0.11455614447822585
4 -0.020248188643009923
5 0.02848986294792888


In [77]:
#tabela_dados3=np.array(tabela_dados3.iloc[:,[2,3]])

variaveis=tabela_dados3.shape[1]
concat=np.array(tabela_dados3)
concat=concat.reshape(-1,variaveis,1)
concat;

In [78]:
def norm(x):
    return (x-np.min(x))/(np.max(x)-np.min(x))

x_train0=np.array(norm(np.array(vetor_atendente_final.iloc[:,0:-1]))).reshape(-1,20,20,1)
x_train1=np.array(norm(np.array(vetor_cliente_final.iloc[:,0:-1]))).reshape(-1,20,20,1)
x_train1.shape

(2771, 20, 20, 1)

In [79]:
from sklearn.model_selection import train_test_split

X, y = x_train0, target
X1, y1 = x_train1, target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)
X_train1, X_test1, y_train1, y_test1 = train_test_split(X1, y1, test_size=0.1, random_state=42)
concat_train,concat_test, y_train2, y_test2=train_test_split(concat, y1, test_size=0.1, random_state=42)

In [80]:
from keras.layers import Input, Dense, Lambda, Concatenate
from keras.models import Model
from keras import backend as K
from keras import objectives
from keras.datasets import mnist
from keras.layers.core import Reshape
import numpy as np
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D, UpSampling2D,Convolution1D,MaxPooling1D
from keras.utils import np_utils
from keras.layers.normalization import BatchNormalization
from keras.callbacks import ModelCheckpoint,LearningRateScheduler
import os
from keras.optimizers import SGD
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
from keras.callbacks import EarlyStopping,ReduceLROnPlateau,ModelCheckpoint

config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)
#tf.reset_default_graph()

#graph = tf.Graph()

with session.as_default():
     with session.graph.as_default():
        
        
        

        input_shape2=(20,20,1)
        input_img = Input(batch_shape=(None, 20,20,1))
        input_img1 = Input(batch_shape=(None, 20,20,1))
        input_img2 = Input(batch_shape=(None, variaveis,1))


        denoise_left=Convolution2D(20, 3,3,
                                border_mode='same',
                                input_shape=input_shape)(input_img)
        denoise_left=BatchNormalization()(denoise_left)
        denoise_left=Activation('relu')(denoise_left)
        denoise_left=MaxPooling2D(pool_size=(2,2))(denoise_left)
        denoise_left=Convolution2D(20, 3, 3,
                                    init='glorot_uniform',border_mode='same')(denoise_left)
        denoise_left=BatchNormalization()(denoise_left)
        denoise_left=Activation('relu')(denoise_left)
        denoise_left=Convolution2D(8, 3, 3,init='glorot_uniform',border_mode='same')(denoise_left)
        denoise_left=BatchNormalization()(denoise_left)
        denoise_left=Activation('relu')(denoise_left)
        denoise_left=UpSampling2D(size=(2, 2))(denoise_left)
        denoise_left=Convolution2D(8, 3, 3,init='glorot_uniform',border_mode='same')(denoise_left)
        denoise_left=BatchNormalization()(denoise_left)
        denoise_left=Activation('relu')(denoise_left)
        denoise_left=UpSampling2D(size=(2, 2))(denoise_left)
        denoise_left=Convolution2D(8, 3, 3,init='glorot_uniform',border_mode='same')(denoise_left)
        denoise_left=BatchNormalization()(denoise_left)
        denoise_left=Activation('relu')(denoise_left)
        denoise_left=Convolution2D(1, 3, 3,init='glorot_uniform',border_mode='same')(denoise_left)
        denoise_left=BatchNormalization()(denoise_left)
        denoise_left=Activation('relu')(denoise_left)
        denoise_left=Dense(20)(denoise_left)
        denoise_left=Reshape((-1,20,1))(denoise_left)
        denoise_left=Flatten()(denoise_left)

        denoise_left1=Convolution2D(20, 3,3,
                                border_mode='same',
                                input_shape=input_shape)(input_img1)
        denoise_left1=BatchNormalization()(denoise_left1)
        denoise_left1=Activation('relu')(denoise_left1)
        denoise_left1=MaxPooling2D(pool_size=(2,2))(denoise_left1)
        denoise_left1=Convolution2D(20, 3, 3,
                                    init='glorot_uniform',border_mode='same')(denoise_left1)
        denoise_left1=BatchNormalization()(denoise_left1)
        denoise_left1=Activation('relu')(denoise_left1)
        denoise_left1=Convolution2D(8, 3, 3,init='glorot_uniform',border_mode='same')(denoise_left1)
        denoise_left1=BatchNormalization()(denoise_left1)
        denoise_left1=Activation('relu')(denoise_left1)
        denoise_left1=UpSampling2D(size=(2, 2))(denoise_left1)
        denoise_left1=Convolution2D(8, 3, 3,init='glorot_uniform',border_mode='same')(denoise_left1)
        denoise_left1=BatchNormalization()(denoise_left1)
        denoise_left1=Activation('relu')(denoise_left1)
        denoise_left1=UpSampling2D(size=(2, 2))(denoise_left1)
        denoise_left1=Convolution2D(8, 3, 3,init='glorot_uniform',border_mode='same')(denoise_left1)
        denoise_left1=BatchNormalization()(denoise_left1)
        denoise_left1=Activation('relu')(denoise_left1)
        denoise_left1=Convolution2D(1, 3, 3,init='glorot_uniform',border_mode='same')(denoise_left1)
        denoise_left1=BatchNormalization()(denoise_left1)
        denoise_left1=Activation('relu')(denoise_left1)
        denoise_left1=Dense(20)(denoise_left1)
        denoise_left1=Reshape((-1,20,1))(denoise_left1)
        denoise_left1=Flatten()(denoise_left1)

        denoise_concat=Dense(20)(input_img2)
        denoise_concat=Activation('relu')(denoise_concat)
        denoise_concat=Dense(20)(denoise_concat)
        denoise_concat=Activation('relu')(denoise_concat)
        denoise_concat=Dense(variaveis)(denoise_concat)
        denoise_concat=Reshape((-1,variaveis,1))(denoise_concat)
        denoise_concat=Flatten()(denoise_concat)

        squeeze0=Concatenate()([denoise_left,denoise_left1,denoise_concat])
        squeeze0=Dropout(0.1)(squeeze0)
        squeeze0=Dense(20)(squeeze0)
        squeeze0=Activation('relu')(squeeze0)
        squeeze0=Dense(2)(squeeze0)
        squeeze0=Activation('sigmoid')(squeeze0)

        model = Model(inputs = [input_img,input_img1,input_img2], outputs = squeeze0)

        model.compile(loss='categorical_crossentropy', optimizer=SGD(lr=0.01,momentum=0.6, decay=5e-5, nesterov=False),metrics = ['accuracy'])
        save = ModelCheckpoint('pesos_keras.hdf5', save_best_only=True, monitor='val_loss', mode='min')

        init = tf.group(tf.global_variables_initializer(), tf.local_variables_initializer())
        session.run(init)
        model.fit([X_train, X_train1,concat_train],y_train,
                nb_epoch=600,
                batch_size=2771,verbose=1,validation_split=0.1,callbacks=[save]
                 )
        preds=model.predict([X_test,X_test1,concat_test])

/home/rubensvectomobile_gmail_com/.local/lib/python3.5/site-packages/tensorflow/python/client/session.py:1735: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:43: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(20, (3, 3), input_shape=(20, 20, 1..., padding="same")`
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:48: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(20, (3, 3), padding="same", kernel_initializer="glorot_uniform")`
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:51: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(8, (3, 3), padding="same", kernel_initializer="glorot_uniform")`
/usr/local/lib/python3.5/

Train on 2243 samples, validate on 250 samples
Epoch 1/600
2243/2243 [==============================] - 3s 1ms/step - loss: 0.6910 - acc: 0.5363 - val_loss: 0.6786 - val_acc: 0.5800
Epoch 2/600
2243/2243 [==============================] - 0s 82us/step - loss: 0.6713 - acc: 0.6068 - val_loss: 0.6813 - val_acc: 0.5800
Epoch 3/600
2243/2243 [==============================] - 0s 78us/step - loss: 0.6662 - acc: 0.6068 - val_loss: 0.6848 - val_acc: 0.5800
Epoch 4/600
2243/2243 [==============================] - 0s 76us/step - loss: 0.6670 - acc: 0.6068 - val_loss: 0.6837 - val_acc: 0.5800
Epoch 5/600
2243/2243 [==============================] - 0s 76us/step - loss: 0.6662 - acc: 0.6068 - val_loss: 0.6803 - val_acc: 0.5800
Epoch 6/600
2243/2243 [==============================] - 0s 76us/step - loss: 0.6633 - acc: 0.6068 - val_loss: 0.6773 - val_acc: 0.5800
Epoch 7/600
2243/2243 [==============================] - 0s 77us/step - loss: 0.6615 - acc: 0.6068 - val_loss: 0.6755 - val_acc: 0.5800
Ep

Epoch 61/600
2243/2243 [==============================] - 0s 76us/step - loss: 0.4448 - acc: 0.8012 - val_loss: 1.4710 - val_acc: 0.5920
Epoch 62/600
2243/2243 [==============================] - 0s 76us/step - loss: 0.4408 - acc: 0.8092 - val_loss: 1.3784 - val_acc: 0.5960
Epoch 63/600
2243/2243 [==============================] - 0s 76us/step - loss: 0.4373 - acc: 0.8003 - val_loss: 1.0625 - val_acc: 0.6120
Epoch 64/600
2243/2243 [==============================] - 0s 76us/step - loss: 0.4290 - acc: 0.8136 - val_loss: 0.9697 - val_acc: 0.6200
Epoch 65/600
2243/2243 [==============================] - 0s 76us/step - loss: 0.4262 - acc: 0.8208 - val_loss: 0.9325 - val_acc: 0.6240
Epoch 66/600
2243/2243 [==============================] - 0s 76us/step - loss: 0.4200 - acc: 0.8194 - val_loss: 0.8610 - val_acc: 0.6440
Epoch 67/600
2243/2243 [==============================] - 0s 76us/step - loss: 0.4166 - acc: 0.8217 - val_loss: 0.8701 - val_acc: 0.6360
Epoch 68/600
2243/2243 [=================

Epoch 121/600
2243/2243 [==============================] - 0s 76us/step - loss: 0.2133 - acc: 0.9487 - val_loss: 0.7322 - val_acc: 0.6600
Epoch 122/600
2243/2243 [==============================] - 0s 76us/step - loss: 0.2150 - acc: 0.9416 - val_loss: 0.7980 - val_acc: 0.6640
Epoch 123/600
2243/2243 [==============================] - 0s 76us/step - loss: 0.2203 - acc: 0.9309 - val_loss: 0.7486 - val_acc: 0.6480
Epoch 124/600
2243/2243 [==============================] - 0s 76us/step - loss: 0.2326 - acc: 0.9215 - val_loss: 0.9449 - val_acc: 0.6760
Epoch 125/600
2243/2243 [==============================] - 0s 76us/step - loss: 0.2711 - acc: 0.8671 - val_loss: 0.7878 - val_acc: 0.6280
Epoch 126/600
2243/2243 [==============================] - 0s 76us/step - loss: 0.3238 - acc: 0.8373 - val_loss: 1.4428 - val_acc: 0.6080
Epoch 127/600
2243/2243 [==============================] - 0s 76us/step - loss: 0.4131 - acc: 0.7637 - val_loss: 0.7513 - val_acc: 0.6640
Epoch 128/600
2243/2243 [=========

Epoch 181/600
2243/2243 [==============================] - 0s 76us/step - loss: 0.0590 - acc: 0.9973 - val_loss: 0.9941 - val_acc: 0.6160
Epoch 182/600
2243/2243 [==============================] - 0s 76us/step - loss: 0.0587 - acc: 0.9964 - val_loss: 0.9950 - val_acc: 0.6200
Epoch 183/600
2243/2243 [==============================] - 0s 76us/step - loss: 0.0558 - acc: 0.9978 - val_loss: 0.9999 - val_acc: 0.6200
Epoch 184/600
2243/2243 [==============================] - 0s 76us/step - loss: 0.0552 - acc: 0.9978 - val_loss: 1.0005 - val_acc: 0.6200
Epoch 185/600
2243/2243 [==============================] - 0s 76us/step - loss: 0.0541 - acc: 0.9973 - val_loss: 1.0032 - val_acc: 0.6200
Epoch 186/600
2243/2243 [==============================] - 0s 76us/step - loss: 0.0527 - acc: 0.9982 - val_loss: 1.0115 - val_acc: 0.6200
Epoch 187/600
2243/2243 [==============================] - 0s 76us/step - loss: 0.0519 - acc: 0.9982 - val_loss: 1.0151 - val_acc: 0.6200
Epoch 188/600
2243/2243 [=========

...

Epoch 301/600
2243/2243 [==============================] - 0s 76us/step - loss: 0.0095 - acc: 1.0000 - val_loss: 1.3979 - val_acc: 0.6280
Epoch 302/600
2243/2243 [==============================] - 0s 76us/step - loss: 0.0093 - acc: 1.0000 - val_loss: 1.4003 - val_acc: 0.6280
Epoch 303/600
2243/2243 [==============================] - 0s 77us/step - loss: 0.0092 - acc: 1.0000 - val_loss: 1.4024 - val_acc: 0.6320
Epoch 304/600
2243/2243 [==============================] - 0s 76us/step - loss: 0.0090 - acc: 1.0000 - val_loss: 1.4061 - val_acc: 0.6400
Epoch 305/600
2243/2243 [==============================] - 0s 77us/step - loss: 0.0089 - acc: 1.0000 - val_loss: 1.4114 - val_acc: 0.6400
Epoch 306/600
2243/2243 [==============================] - 0s 76us/step - loss: 0.0092 - acc: 1.0000 - val_loss: 1.4089 - val_acc: 0.6400
Epoch 307/600
2243/2243 [==============================] - 0s 76us/step - loss: 0.0088 - acc: 1.0000 - val_loss: 1.4122 - val_acc: 0.6400
Epoch 308/600
2243/2243 [=========

Epoch 361/600
2243/2243 [==============================] - 0s 76us/step - loss: 0.0058 - acc: 1.0000 - val_loss: 1.5340 - val_acc: 0.6400
Epoch 362/600
2243/2243 [==============================] - 0s 76us/step - loss: 0.0057 - acc: 1.0000 - val_loss: 1.5363 - val_acc: 0.6400
Epoch 363/600
2243/2243 [==============================] - 0s 76us/step - loss: 0.0056 - acc: 1.0000 - val_loss: 1.5380 - val_acc: 0.6400
Epoch 364/600
2243/2243 [==============================] - 0s 76us/step - loss: 0.0056 - acc: 1.0000 - val_loss: 1.5408 - val_acc: 0.6400
Epoch 365/600
2243/2243 [==============================] - 0s 76us/step - loss: 0.0055 - acc: 1.0000 - val_loss: 1.5427 - val_acc: 0.6400
Epoch 366/600
2243/2243 [==============================] - 0s 76us/step - loss: 0.0055 - acc: 1.0000 - val_loss: 1.5467 - val_acc: 0.6400
Epoch 367/600
2243/2243 [==============================] - 0s 77us/step - loss: 0.0056 - acc: 1.0000 - val_loss: 1.5507 - val_acc: 0.6520
Epoch 368/600
2243/2243 [=========

Epoch 421/600
2243/2243 [==============================] - 0s 76us/step - loss: 0.0038 - acc: 1.0000 - val_loss: 1.6553 - val_acc: 0.6560
Epoch 422/600
2243/2243 [==============================] - 0s 76us/step - loss: 0.0040 - acc: 1.0000 - val_loss: 1.6601 - val_acc: 0.6560
Epoch 423/600
2243/2243 [==============================] - 0s 76us/step - loss: 0.0040 - acc: 1.0000 - val_loss: 1.6580 - val_acc: 0.6560
Epoch 424/600
2243/2243 [==============================] - 0s 76us/step - loss: 0.0039 - acc: 1.0000 - val_loss: 1.6582 - val_acc: 0.6560
Epoch 425/600
2243/2243 [==============================] - 0s 76us/step - loss: 0.0039 - acc: 1.0000 - val_loss: 1.6615 - val_acc: 0.6560
Epoch 426/600
2243/2243 [==============================] - 0s 76us/step - loss: 0.0038 - acc: 1.0000 - val_loss: 1.6623 - val_acc: 0.6560
Epoch 427/600
2243/2243 [==============================] - 0s 77us/step - loss: 0.0038 - acc: 1.0000 - val_loss: 1.6622 - val_acc: 0.6560
Epoch 428/600
2243/2243 [=========

Epoch 481/600
2243/2243 [==============================] - 0s 76us/step - loss: 0.0029 - acc: 1.0000 - val_loss: 1.7400 - val_acc: 0.6600
Epoch 482/600
2243/2243 [==============================] - 0s 76us/step - loss: 0.0029 - acc: 1.0000 - val_loss: 1.7409 - val_acc: 0.6600
Epoch 483/600
2243/2243 [==============================] - 0s 76us/step - loss: 0.0028 - acc: 1.0000 - val_loss: 1.7417 - val_acc: 0.6600
Epoch 484/600
2243/2243 [==============================] - 0s 77us/step - loss: 0.0029 - acc: 1.0000 - val_loss: 1.7459 - val_acc: 0.6600
Epoch 485/600
2243/2243 [==============================] - 0s 76us/step - loss: 0.0029 - acc: 1.0000 - val_loss: 1.7393 - val_acc: 0.6600
Epoch 486/600
2243/2243 [==============================] - 0s 76us/step - loss: 0.0029 - acc: 1.0000 - val_loss: 1.7435 - val_acc: 0.6600
Epoch 487/600
2243/2243 [==============================] - 0s 76us/step - loss: 0.0029 - acc: 1.0000 - val_loss: 1.7465 - val_acc: 0.6600
Epoch 488/600
2243/2243 [=========

Epoch 541/600
2243/2243 [==============================] - 0s 76us/step - loss: 0.0023 - acc: 1.0000 - val_loss: 1.8051 - val_acc: 0.6560
Epoch 542/600
2243/2243 [==============================] - 0s 76us/step - loss: 0.0022 - acc: 1.0000 - val_loss: 1.8047 - val_acc: 0.6600
Epoch 543/600
2243/2243 [==============================] - 0s 76us/step - loss: 0.0022 - acc: 1.0000 - val_loss: 1.8037 - val_acc: 0.6600
Epoch 544/600
2243/2243 [==============================] - 0s 76us/step - loss: 0.0023 - acc: 1.0000 - val_loss: 1.8052 - val_acc: 0.6600
Epoch 545/600
2243/2243 [==============================] - 0s 76us/step - loss: 0.0023 - acc: 1.0000 - val_loss: 1.8068 - val_acc: 0.6600
Epoch 546/600
2243/2243 [==============================] - 0s 76us/step - loss: 0.0023 - acc: 1.0000 - val_loss: 1.8096 - val_acc: 0.6600
Epoch 547/600
2243/2243 [==============================] - 0s 76us/step - loss: 0.0022 - acc: 1.0000 - val_loss: 1.8097 - val_acc: 0.6560
Epoch 548/600
2243/2243 [=========

In [81]:
from keras.utils import plot_model 
plot_model(model, to_file='model_gif.png')

In [82]:
preds1=preds

In [83]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score,precision_score,confusion_matrix,f1_score,recall_score


In [84]:
confusion_matrix(y_test.T[1],np.argmax(preds,axis=1))
pred0=np.argmax(preds,axis=1)
Y=y_test.T[1]
print('\n')
print('Accuracy:',accuracy_score(np.array(Y),pred0))
print('Precision (FP):',precision_score(np.array(Y),pred0,average='binary'))
print('Recall (FN):',recall_score(np.array(Y),pred0,average='binary'))
print('f1:',f1_score(np.array(Y),pred0,average='binary'))
print(confusion_matrix(np.array(Y),pred0),'\n')



Accuracy: 0.7194244604316546
Precision (FP): 0.7666666666666667
Recall (FN): 0.7931034482758621
f1: 0.7796610169491527
[[ 62  42]
 [ 36 138]] 

